## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Alfred,ZA,2022-07-16 19:10:16,-33.5906,26.8910,67.30,53,0,8.30,clear sky
1,1,Kodinsk,RU,2022-07-16 19:10:16,58.6881,99.1844,51.82,85,14,1.23,few clouds
2,2,Huarmey,PE,2022-07-16 19:10:16,-10.0681,-78.1522,66.99,71,5,14.41,clear sky
3,3,Salalah,OM,2022-07-16 19:09:00,17.0151,54.0924,78.80,90,100,4.68,overcast clouds
4,4,Olafsvik,IS,2022-07-16 19:10:17,64.8945,-23.7142,50.49,92,100,1.99,light rain


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
user_min_temp = float(input("What is the minimum temperature you would like for your trip? "))
user_max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= user_max_temp) & 
                                       (city_data_df["Max Temp"] >= user_min_temp)]

In [20]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isna().sum()

City_ID                0
City                   0
Country                1
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [21]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna(0,'any') 
preferred_cities_df.isna().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [22]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Salalah,OM,78.80,overcast clouds,17.0151,54.0924,
6,Asau,RO,74.32,clear sky,46.4333,26.4000,
8,Hilo,US,84.27,moderate rain,19.7297,-155.0900,
14,Rikitea,PF,73.33,light rain,-23.1203,-134.9692,
15,Avarua,CK,75.25,few clouds,-21.2078,-159.7750,
19,Maumere,ID,76.33,scattered clouds,-8.6199,122.2111,
20,Ribeira Grande,PT,77.40,few clouds,38.5167,-28.7000,
23,Rodrigues Alves,BR,84.18,overcast clouds,-7.7380,-72.6509,
25,Inongo,CD,76.64,overcast clouds,-1.9500,18.2667,
26,Kavaratti,IN,81.82,overcast clouds,10.5669,72.6420,


In [48]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for i, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    latit = row["Lat"]
    longit = row["Lng"]
    params["location"] = f"{latit},{longit}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[i, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel could not be found.... skipping")
print("Done!")     

Hotel could not be found.... skipping
Hotel could not be found.... skipping
Hotel could not be found.... skipping
Hotel could not be found.... skipping
Hotel could not be found.... skipping
Hotel could not be found.... skipping
Hotel could not be found.... skipping
Hotel could not be found.... skipping
Hotel could not be found.... skipping
Hotel could not be found.... skipping
Hotel could not be found.... skipping
Hotel could not be found.... skipping
Hotel could not be found.... skipping
Hotel could not be found.... skipping
Hotel could not be found.... skipping
Hotel could not be found.... skipping
Hotel could not be found.... skipping
Hotel could not be found.... skipping
Hotel could not be found.... skipping
Hotel could not be found.... skipping
Hotel could not be found.... skipping


In [55]:
hotel_df.isna().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [56]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(0,"any")

In [57]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file)

In [59]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [60]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))